## Clonagem de Voz em Português

Este notebook demonstra como gerar voz clonada em português usando OpenVoice.


In [ ]:
import os
# Garantir que estamos no diretório correto
expected_dir = '/home/mateuus/projects/eopix/openvoice-server'
if os.getcwd() != expected_dir:
    print(f"Mudando diretório de {os.getcwd()} para {expected_dir}")
    os.chdir(expected_dir)
print(f"Diretório atual: {os.getcwd()}")

import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter
from openai import OpenAI
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()


### Configuração


In [ ]:
# Configurações
ckpt_converter = 'checkpoints/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

print(f"Device: {device}")
print(f"Output directory: {output_dir}")

# Carregar o conversor de cor de tom
tone_color_converter = ToneColorConverter(
    f'{ckpt_converter}/config.json', 
    device=device
)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)
print("✓ Conversor carregado com sucesso")


### Abordagem para Português

Para gerar voz em português com OpenVoice, usamos **OpenAI TTS** como base speaker.

O OpenVoice funciona assim:
1. Gera o áudio com um base speaker (pode ser em qualquer idioma)
2. Aplica a cor de tom (timbre) da voz de referência

Para português, usamos:
- **Base speaker**: OpenAI TTS em português
- **Cor de tom**: Qualquer voz de referência que você quiser clonar


### 1. Gerar base speaker usando OpenAI TTS


In [ ]:
# Inicializar cliente OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Gerar um áudio em português usando OpenAI TTS
texto_base = "Olá, este é um teste de clonagem de voz em português brasileiro."
print(f"Gerando áudio base em português: {texto_base}")

response = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input=texto_base
)

base_speaker_path = f"{output_dir}/base_pt.wav"
response.stream_to_file(base_speaker_path)
print(f"✓ Base speaker gerado: {base_speaker_path}")


### 2. Extrair embeddings de cor de tom


In [ ]:
# Extrair embedding do base speaker (áudio gerado pelo OpenAI TTS)
print("Extraindo embedding do base speaker...")
source_se, audio_name = se_extractor.get_se(
    base_speaker_path, 
    tone_color_converter, 
    vad=True
)
print(f"✓ Source SE extraído: {audio_name}")


In [ ]:
# Extrair embedding da voz de referência
# Use um arquivo de áudio de 3-10 segundos com a voz que você quer clonar
reference_speaker = 'resources/example_reference.mp3'  # Troque pelo seu arquivo

print(f"Extraindo embedding da voz de referência: {reference_speaker}")
target_se, audio_name = se_extractor.get_se(
    reference_speaker, 
    tone_color_converter, 
    vad=True
)
print(f"✓ Target SE extraído: {audio_name}")


### 3. Gerar voz clonada em português


In [ ]:
# Textos para gerar em português
textos_pt = [
    "Olá, este é o OpenVoice, um sistema de clonagem de voz.",
    "Ele pode replicar a cor tonal de qualquer voz de referência.",
    "É possível gerar fala em múltiplos idiomas e com diferentes emoções."
]

src_path = f'{output_dir}/tmp_pt.wav'

for i, texto in enumerate(textos_pt):
    print(f"\n[{i+1}/{len(textos_pt)}] Processando: {texto}")
    
    # 1. Gerar TTS em português com OpenAI
    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=texto
    )
    response.stream_to_file(src_path)
    
    # 2. Aplicar clonagem de voz
    save_path = f'{output_dir}/output_pt_{i}.wav'
    
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path,
        src_se=source_se,
        tgt_se=target_se,
        output_path=save_path,
        message=encode_message
    )
    
    print(f"✓ Arquivo gerado: {save_path}")

print("\n🎉 Todos os arquivos foram gerados com sucesso!")
print(f"📂 Verifique os arquivos em: {output_dir}/")


### 📝 Notas Importantes

#### ❌ Você NÃO precisa criar checkpoints
Os arquivos necessários já existem:
- `checkpoints_v2/base_speakers/ses/en-br.pth` - Português Brasileiro
- `checkpoints/converter/checkpoint.pth` - Conversor de cor de tom

#### ✅ Como funciona a clonagem
1. **Base Speaker**: Gera o conteúdo de fala (texto → áudio)
   - Usamos OpenAI TTS para português
2. **Conversor de Cor de Tom**: Aplica o timbre da voz de referência
   - Captura a voz de referência e aplica no base speaker

#### 🔧 Ajustar para sua voz
Troque o arquivo `reference_speaker` pelo seu áudio:
- 3-10 segundos de áudio
- Fala clara e limpa
- Qualquer formato (mp3, wav, etc)

#### 🚀 Melhorias futuras
Para melhor qualidade, instale o MeloTTS:
```bash
pip install melo-tts
```
Depois use o arquivo `en-br.pth` conforme demonstrado no `demo_part3.ipynb`
